In [1]:
import pandas as pd
import numpy as np
import pathlib 
import os
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm import tqdm
import warnings
import pandas as pd
from sklearn.metrics import roc_curve, auc
from scipy.stats import pearsonr

warnings.filterwarnings('ignore')

data_version = "TMD_cov"
output_version = "20240910"

outdir = "/media/hieunguyen/GSHD_HN01/outdir"
PROJECT = "TMD450_TCGA_data_analysis"
thres_hypo = 0.3
thres_hyper = 0.6


for mode in ["all", "hypo_only", "hyper_only"]:
    all_cancer_classes = ["Liver", "Lung", "Breast", "CRC"]
    thresdf = pd.read_csv(f"thresdf.{mode}.csv")

    for input_cancer_class in all_cancer_classes:
        path_to_main_output = os.path.join(outdir, PROJECT, output_version)

        if mode == "all":
            path_to_13_output = os.path.join(outdir, PROJECT, output_version, "13_output", input_cancer_class, "thres_hypo_{}_hyper_{}".format(thres_hypo, thres_hyper))
            path_to_14_output = os.path.join(outdir, PROJECT, output_version, "14_output", input_cancer_class, "thres_hypo_{}_hyper_{}".format(thres_hypo, thres_hyper))
        elif mode == "hypo_only":
            path_to_13_output = os.path.join(outdir, PROJECT, output_version, "13_output_all_hypo", input_cancer_class, "thres_hypo_{}_hyper_{}".format(thres_hypo, thres_hyper))
            path_to_14_output = os.path.join(outdir, PROJECT, output_version, "14_output_all_hypo", input_cancer_class, "thres_hypo_{}_hyper_{}".format(thres_hypo, thres_hyper))
        elif mode == "hyper_only":
            path_to_13_output = os.path.join(outdir, PROJECT, output_version, "13_output_all_hyper", input_cancer_class, "thres_hypo_{}_hyper_{}".format(thres_hypo, thres_hyper))
            path_to_14_output = os.path.join(outdir, PROJECT, output_version, "14_output_all_hyper", input_cancer_class, "thres_hypo_{}_hyper_{}".format(thres_hypo, thres_hyper))

        os.system(f"mkdir -p {path_to_14_output}")

        all_files = [item for item in pathlib.Path(path_to_13_output).glob("*.candi_reads.csv")]
        countdf = pd.read_csv(os.path.join(path_to_13_output, "all_count.csv"), index_col = [0])

        all_samples = []
        all_mean_candi_reads = []
        for file in all_files:
            tmpdf = pd.read_csv(file)
            mean_candi_reads = tmpdf.candi_reads.mean()
            all_samples.append(file.name.split(".")[0])
            all_mean_candi_reads.append(mean_candi_reads)

        candidf = pd.DataFrame({"SampleID": all_samples, "num_candi_reads": all_mean_candi_reads})
        candidf = candidf.merge(countdf, right_on = "SampleID", left_on = "SampleID")
        candidf["ratio_raw"] = candidf["num_candi_reads"] / candidf["raw_count"]
        candidf["ratio_in_reads"] = candidf["num_candi_reads"] / candidf["in_read_count"]

        
        candidf.to_excel(os.path.join(path_to_14_output, "candi_reads_all_{}_vs_control.xlsx".format(input_cancer_class)), index = False)

        
